<br>
<br>
<center> <font size = 6> Importance Sampling: Statistics on Rare Events </font> </center>
<br>
<br>
<center> <font size = 3> Last Updated: 20/01-2023 </font> </center>

### Acknowledgements

This notebook is created based on the tutorial by QuantPy. The video can be found here: https://www.youtube.com/watch?v=V8kruQjqpuw&t=921s. 

### Imports

In [2]:
import numpy as np
import time
import math
import scipy.stats

### What is importance sampling? 

At times, we wish to sample rare events from a known distribution. However, getting the right realizations might take many simulations. To reduce the computational demand, we can change the measure to a equivalent one, where we need fewer repetitions. 

Consider the probability space $(\Omega, \mathcal{F}, P)$, where $\Omega$ is the sample space (i.e., set of all outcomes), $\mathcal{F}$ is a $\sigma$-algebra (i.e., set of all events we wish to consider), and $P : \mathcal{F} \mapsto [0, 1]$ is a probability measure which assigns a probability to each considered event.

**Definition:** $\sigma$-algebra

Consider the set $\Omega$. Then a subset $\Sigma \subseteq \mathcal{P}(\Omega)$, where $\mathcal{P}(\Omega)$ is the power set of $\Omega$, is a $\sigma$-algebra if it satisfies

1. $\Omega \in \Sigma$ (i.e., the set contains itself)
2. $\Sigma$ is closed under complementation (i.e., if $A \in \Sigma$, then $(\Omega - A) \in \Sigma$)
3. $\Sigma$ is closed under countable union (i.e., for any sets $A_{i} \in \Sigma$, then $\cup A_{i} \in \Sigma$)

**Outline of Theory**

Suppose we want to find $\mathbb{E}[f(X)] = \int_{\Omega} f(X) dP$. 

Introducing a simple fraction, we get the simplification

$$
\mathbb{E}[f(X)] = \int_{\Omega} f(X) \frac{dP}{dQ}dQ = \mathbb{E}^{Q}[f(X) \frac{dP}{dQ}] \approx \frac{1}{n} \sum_{i=1}^{n} f(x_{i}) \frac{p(x_{i})}{q(x_{i})}
$$

which means that we can find the expectation $\mathbb{E}[f(X)]$ w.r.t. to the measure $\mathbb{P}$ by finding the expectation of $f(X)$ multiplied with the above fraction under an equivalent measure $Q$.

**Example 1**

Suppose we want to find $\mathbb{E}[I_{X \geq 25}] = P(X \geq 25)$. Let $X \sim N(0, 1)$ (i.e., the *standard normal distribution*).

In [3]:
start_time = time.time()
x1 = np.random.normal(loc = 0, scale = 1, size = 10000)
end_time = time.time()
print("It took {} seconds to draw the RVs".format(round(end_time - start_time, 3)))

It took 0.003 seconds to draw the RVs


In [4]:
np.sum(x1 >= 25)

0

With 10.000 drawn random variables, none exceed 25. It is because it corresponds to a 25-$\sigma$ event, which is very unlikely.

In [5]:
start_time = time.time()
x2 = np.random.normal(loc = 0, scale = 1, size = 10**(9))
end_time = time.time()
print("It took {} seconds to draw the RVs.".format(round(end_time - start_time, 3)))

It took 15.896 seconds to draw the RVs.


In [6]:
np.sum(x2 >= 25)

0

Even with $10^{9}$ drawn random variables (took approximately 16.8 seconds), none exceed 25. This stresses the need for importance sampling. Recall the definition of the probability density function of the normal distribution.

$$
f(x) = \frac{1}{\sqrt{2\pi \sigma^{2}}} \exp{(-\frac{(x - \mu)^{2}}{2 \sigma^{2}})}
$$

Now we introduce $p(x)$ (based on $\mu = 0$ and $\sigma = 1$) and $q(x)$ ($\mu$ unspecified and $\sigma = 1$).

$$
p(x) = \frac{1}{\sqrt{2\pi}} \exp{(-\frac{x^{2}}{2})} \ \ \ \text{and} \ \ \ q(x) = \frac{1}{\sqrt{2\pi}} \exp{(-\frac{(x - \mu)^{2}}{2})}
$$

Then we find the ratio.

$$
\frac{p(x)}{q(x)} = \frac{\frac{1}{\sqrt{2\pi}} \exp{(-\frac{x^{2}}{2})}}{\frac{1}{\sqrt{2\pi}} \exp{(-\frac{(x - \mu)^{2}}{2})}} = \exp{(-\frac{x^{2}}{2} + \frac{(x - \mu)^{2}}{2})} = \exp{(\frac{\mu^{2}}{2} - x \mu)}
$$

Let $\tilde X \sim N(0, 1)$ and $X \sim N(25, 1)$.

$$
\mathbb{E}[I_{\tilde X \geq 25}] = \int_{\Omega} I_{X \geq 25} \exp{(\frac{\mu^{2}}{2} - x \mu)} \ \ dQ  = \mathbb{E}^{Q}[I_{X \geq 25} \exp{(\frac{\mu^{2}}{2} - x \mu)}] \approx \frac{1}{n} \sum_{i=1}^{n} I_{X \geq 25} \exp{(\frac{\mu^{2}}{2} - x \mu)}
$$

In [7]:
def f(x, mu = 0, sigma = 1):
    return(1/np.sqrt(2*math.pi*(sigma**2)) * np.exp(-((x - mu)**2)/(2*sigma**2)))
    
def p(x):
    return(f(x))

def q(x, mu = 25):
    return(f(x, mu = mu, sigma = 1))

Then we can sample again but with the new distribution.

In [10]:
mu = 25

start_time = time.time()
x3 = np.random.normal(loc = mu, scale = 1, size = 10**(5))
end_time = time.time()
print("It took {} seconds to draw the RVs.".format(round(end_time - start_time, 3)))

start_time = time.time()
STAT = (x3 >= mu) * p(x3)/q(x3)
E = np.mean(STAT)
SD = np.std(STAT)
SE = SD/np.sqrt(len(x3))

dist = scipy.stats.norm(loc = 0, scale = 1)
confidence = 0.05

print("\nExpectation: {}".format(E))
print("Standard Deviation: {}".format(SD))
print("Confidence Interval: [{}, {}]\n".format(E + SE * dist.ppf(confidence/2), E + SE * dist.ppf(1 - confidence/2)))
end_time = time.time()
print("It took {} seconds to find the expectation.".format(round(end_time - start_time, 3)))

It took 0.006 seconds to draw the RVs.

Expectation: 3.0030743702474874e-138
Standard Deviation: 1.67869185528672e-137
Confidence Interval: [2.8990298829798034e-138, 3.107118857515171e-138]

It took 0.007 seconds to find the expectation.
